<a href="https://colab.research.google.com/github/den-niz/Shopbot/blob/main/Shopbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Installing packages
!pip install --quiet openai python-dotenv
!pip install comet_ml

In [12]:
# Importing nessessary packages
import os
from openai import OpenAI
import comet_ml
from IPython.display import display
import ipywidgets as widgets
import time
import dotenv


In [13]:
!pip install comet_ml

In [14]:
# API keys
MY_OPENAI_KEY = os.getenv("OPENAI_API_KEY")
MY_COMET_KEY = os.getenv("COMET_API_KEY")

comet_ml.login(project_name="ecom_shopbot",
               api_key= "COMET_API_KEY")

COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).


In [15]:
# product list
product_list = '''
# Fashion Shop Product List

## Women's Clothing:

- T-shirt
  - Price: $20
  - Available Sizes: Small, Medium, Large, XL
  - Available Colors: Red, White, Black, Gray, Navy

- Elegant Evening Gown
  - Price: $150
  - Available Sizes: Small, Medium, Large, XL
  - Available Colors: Black, Navy Blue, Burgundy

- Floral Summer Dress
  - Price: $45
  - Available Sizes: Small, Medium, Large
  - Available Colors: Floral Print, Blue, Pink

- Professional Blazer
  - Price: $80
  - Available Sizes: Small, Medium, Large, XL
  - Available Colors: Black, Gray, Navy

## Men's Clothing:
- Classic Suit Set
  - Price: $200
  - Available Sizes: Small, Medium, Large, XL
  - Available Colors: Charcoal Gray, Navy Blue, Black

- Casual Denim Jeans
  - Price: $35
  - Available Sizes: 28, 30, 32, 34
  - Available Colors: Blue Denim, Black

- Polo Shirt Collection
  - Price: $25 each
  - Available Sizes: Small, Medium, Large, XL
  - Available Colors: White, Blue, Red, Green

## Accessories:
- Stylish Sunglasses
  - Price: $20
  - Available Colors: Black, Brown, Tortoise Shell

- Leather Handbag
  - Price: $60
  - Available Colors: Brown, Black, Red

- Classic Wristwatch
  - Price: $50
  - Available Colors: Silver, Gold, Rose Gold

## Footwear:
- High-Heel Ankle Boots
  - Price: $70
  - Available Sizes: 5-10
  - Available Colors: Black, Tan, Burgundy

- Comfortable Sneakers
  - Price: $55
  - Available Sizes: 6-12
  - Available Colors: White, Black, Gray

- Formal Leather Shoes
  - Price: $90
  - Available Sizes: 7-11
  - Available Colors: Brown, Black

## Kids' Collection:
- Cute Cartoon T-shirts
  - Price: $15 each
  - Available Sizes: 2T, 3T, 4T
  - Available Colors: Blue, Pink, Yellow

- Adorable Onesies
  - Price: $25
  - Available Sizes: Newborn, 3M, 6M, 12M
  - Available Colors: Pastel Blue, Pink, Mint Green

- Trendy Kids' Backpacks
  - Price: $30
  - Available Colors: Blue, Red, Purple

## Activewear:
- Yoga Leggings
  - Price: $30
  - Available Sizes: Small, Medium, Large
  - Available Colors: Black, Gray, Teal

- Running Shoes
  - Price: $40
  - Available Sizes: 6-12
  - Available Colors: White, Black, Neon Green

- Quick-Dry Sports T-shirt
  - Price: $20
  - Available Sizes: Small, Medium, Large
  - Available Colors: Red, Blue, Gray


'''

context = [{'role': 'system',
            'content': f"""
You are ShopBot, an AI assistant for my online fashion shop - Denniz Designz.

Your role is to assist customers in browsing products, providing information, and guiding them through the checkout process.

Be friendly and helpful in your interactions.

We offer a variety of products across categories such as Women's Clothing, Men's clothing, Accessories, Kids' Collection, Footwears and Activewear products.

Feel free to ask customers about their preferences, recommend products, and inform them about any ongoing promotions.

The Current Product List is limited as below:

```{product_list}```

Make the shopping experience enjoyable and encourage customers to reach out if they have any questions or need assistance.
"""}]



In [16]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo"):
    client = OpenAI(
        api_key=MY_OPENAI_KEY,
    )

    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
    )
    return chat_completion.choices[0].message.content

In [17]:

def collect_messages(_):
    user_input = inp.value
    inp.value = ''
    context.append({'role':'user', 'content':f"{user_input}"})

    # Record the start time
    start_time = time.time()

    response = get_completion_from_messages(context)

    # Record the end time
    end_time = time.time()

    # Calculate the duration
    duration = end_time - start_time

    # Log to comet_llm
    comet_ml.log_prompt(
        prompt=user_input,
        output=response,
        duration=duration,
        metadata={
        "role": context[-1]['role'],
        "content": context[-1]['content'],
        "context": context,
        "product_list": product_list
        },
    )

    context.append({'role': 'assistant', 'content': f"{response}"})

    user_pane = widgets.Output()
    with user_pane:
        display(widgets.HTML(f"<b>User:</b> {user_input}"))

    assistant_pane = widgets.Output()
    with assistant_pane:
        display(widgets.HTML(f"<b>Assistant:</b> {response}"))

    display(widgets.VBox([user_pane, assistant_pane]))

inp = widgets.Text(value="Hi", placeholder='Enter text here…')
button_conversation = widgets.Button(description="Chat!")
button_conversation.on_click(collect_messages)

dashboard = widgets.VBox([inp, button_conversation])

display(dashboard)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable